In [ ]:
import cv2
import glob
import json
from tqdm import tqdm
import os
import re
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 경로 확인

In [ ]:
# 기본 디렉토리 설정
base_path = '/content/drive/MyDrive/food_data/Training/unzip'

unzip_num_path = glob.glob(os.path.join(base_path, '*'))
sorted_unzip = sorted(unzip_num_path)

# 정규식 패턴 정의 (알파벳 뒤에 오는 숫자가 18, 20, 21, 28인 경우 필터링)
exclude_pattern = re.compile(r'^[a-zA-Z](18|20|21|28)')

for folder in sorted_unzip[-4:-3]: ####
    print('start: ' + folder)

### Unzip 파일 갯수 확인

In [ ]:
# 압축 파일 경로와 해제할 경로 설정
unzip_file_path = '/content/drive/MyDrive/food_data/Training/unzip'

# ZIP 파일들의 경로 리스트를 얻기
unzip_files = glob.glob(os.path.join(unzip_file_path, '*'))
num_files = sorted(unzip_files)

for file_path in num_files[-4:-3]:
  print("\n확인할 폴더: " + file_path)
  img_path = os.path.join(file_path, 'image')
  json_path = os.path.join(file_path, 'json')

  img_file_path = glob.glob(os.path.join(img_path, '*'))
  img_sort_files = sorted(img_file_path)

  print("음식 폴더의 수:", len(img_file_path))

  for file in img_sort_files:
    imgs = glob.glob(os.path.join(file, '*'))
    food_name = file.split('/')[-1]
    food_json = food_name + " json"
    print(f"\t{food_name}의 이미지 수:", len(imgs))

    json_sort_files = os.path.join(json_path, food_json)
    jsons = glob.glob(os.path.join(json_sort_files, '*'))
    print(f"\t{food_name}의 json 수:", len(jsons))

    if len(imgs) != len(jsons):
        print(f"\tWarning: {food_name}의 이미지와 json 파일의 수가 다릅니다.")


### 이미지 Crop 진행

In [ ]:
# 기본 디렉토리 설정
base_path = '/content/drive/MyDrive/food_data/Training/unzip'

unzip_num_path = glob.glob(os.path.join(base_path, '*'))
sorted_unzip = sorted(unzip_num_path)

# 정규식 패턴 정의 (알파벳 뒤에 오는 숫자가 18, 20, 21, 28인 경우 필터링)
exclude_pattern = re.compile(r'^[a-zA-Z](18|20|21|28)')

for folder in sorted_unzip[-5:-4]: ####
    print('start: ' + folder)
    json_base_dir = os.path.join(folder, 'json')
    image_base_dir = os.path.join(folder, 'image')

    output_base_dir = '/content/drive/MyDrive/food_data/Crop' ####
    folder_num = os.path.basename(folder)
    folder_path = os.path.join(output_base_dir, folder_num)

    #하위 디렉터리가 없다면 생성
    os.makedirs(folder_path, exist_ok=True)

    # 목록 들고오기
    json_folders = os.listdir(json_base_dir)
    image_folders = os.listdir(image_base_dir)

    # json 폴더 이름을 image 폴더에서 찾기
    for json_folder in json_folders:  #####
        #JSON 폴더 이름에서 뒤에 붙은 ' json' 제거
        json_folder_name = json_folder.rstrip(' json')
        image_folder_path = os.path.join(image_base_dir, json_folder_name)

        # JSON 폴더와 매칭되는 이미지 폴더가 있는지 확인
        if os.path.exists(image_folder_path):
            json_folder_path = os.path.join(json_base_dir, json_folder)
            output_folder_path = os.path.join(folder_path, json_folder_name)

            # JSON 및 이미지 파일 목록 불러오기
            json_files = glob.glob(os.path.join(json_folder_path, '*'))
            image_files = glob.glob(os.path.join(image_folder_path, '*'))

            if not(exclude_pattern.search(os.path.basename(image_files[0]))):
                os.makedirs(output_folder_path, exist_ok=True)

            print(json_folder_name)

            for image_file in tqdm(image_files[:1000]):  #####
                # 파일명이 정규식 패턴에 맞는지 확인
                if exclude_pattern.search(os.path.basename(image_file)):
                    continue  # 패턴에 맞는 파일은 건너뜀

            # for image_file in image_files:   #####
                image_path = os.path.join(image_folder_path, image_file)
                json_file = (os.path.splitext(os.path.basename(image_file))[0]+'.json')
                json_path = os.path.join(json_folder_path, json_file)

                # # 이미지 읽기
                img_read = Image.open(image_path)
                image = np.array(img_read)
                # image = cv2.imread(image_path)
                img_height, img_width, _ = image.shape

                # JSON 파일 읽기
                with open(json_path, 'r') as f:
                    bounding_boxes = json.load(f)

                # JSON에서 바운딩 박스 정보 가져오기
                for i, bbox in enumerate(bounding_boxes):
                    point_str = bbox["Point(x,y)"]
                    width_ratio = float(bbox["W"])
                    height_ratio = float(bbox["H"])

                    x_ratio, y_ratio = map(float, point_str.split(','))

                    # 상대 좌표를 절대 좌표로 변환
                    x = int(x_ratio * img_width)
                    y = int(y_ratio * img_height)
                    width = int(width_ratio * img_width)
                    height = int(height_ratio * img_height)
                    new_x = int(x - width/2)
                    new_y = int(y - height/2)

                    # 이미지 크롭
                    cropped_image = image[new_y:new_y+height, new_x:new_x+width]

                    # 크롭된 이미지가 유효한지 확인
                    if cropped_image.size == 0:
                        print(f"Skipping empty cropped image: {image_file}")
                        continue

                    cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

                    # 크롭된 이미지 저장 경로 설정 splittext->확장자 분리
                    a = os.path.basename(image_file)
                    cropped_img_path = os.path.join(output_folder_path, f"{a.split('.')[0]}_crop.jpg")

                    # 크롭된 이미지 저장
                    # cv2.imwrite(cropped_img_path, cropped_image)

                    # path = save_path + f"/crop_{filename}"
                    ptype = os.path.splitext(cropped_img_path)[1]
                    ret, img_arr = cv2.imencode(ptype, cropped_image_rgb)

                    if ret:
                        with open(cropped_img_path, mode='w+b') as f:
                            img_arr.tofile(f)
                    # print(cropped_img_path)

                    # # 크롭된 이미지 시각화
                    # plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                    # plt.title(f"Cropped Image {i + 1}")
                    # plt.axis('off')
                    # plt.show()

### 1000개 이후의 사진 갖는 디렉터리 수 확인

In [ ]:
###버전1
import os

# 주어진 루트 디렉토리 경로
root_dir = '/content/drive/MyDrive/food_data/Crop'

# 파일 개수 정보를 저장할 딕셔너리 초기화
file_count_per_folder = {}

# 1000개 이하의 파일을 가진 폴더 리스트 초기화
folders_with_few_files = []

# 파일 개수를 세는 함수
def count_files_in_directory(directory):
    file_count = 0
    for root, dirs, files in os.walk(directory):
        file_count += len(files)
    return file_count

# '101'부터 '110'까지의 폴더를 탐색
for folder_num in range(101, 106):
    target_folder = os.path.join(root_dir, str(folder_num))
    if os.path.exists(target_folder):
        # 하위 폴더 탐색
        for food_folder in os.listdir(target_folder):
            food_folder_path = os.path.join(target_folder, food_folder)
            if os.path.isdir(food_folder_path):
                # 해당 폴더 내의 파일 개수 계산
                file_count = count_files_in_directory(food_folder_path)
                file_count_per_folder[food_folder] = file_count
                # 파일 개수가 1000개 이하인 폴더를 리스트에 추가
                if file_count <= 1000:
                    folders_with_few_files.append((food_folder, file_count))

# 파일 개수 정보 출력
print("파일 개수 정보:")
for folder, count in file_count_per_folder.items():
    print(f"{folder}: {count}개 파일")

# 1000개 이하의 파일을 가진 폴더 정보 출력
print("\n1000개 이하의 파일을 가진 폴더:")
for folder, count in folders_with_few_files:
    print(f"{folder}: {count}개 파일")

# 결과 리스트 확인
print("\n1000개 이하의 파일을 가진 폴더 리스트:")
print(folders_with_few_files)

In [ ]:
###버전2
import os

# 주어진 루트 디렉토리 경로
root_dir = '/content/drive/MyDrive/food_data/Crop'

# 파일 개수 정보를 저장할 딕셔너리 초기화
file_count_per_folder = {}

# 1000개 이하의 파일을 가진 폴더 리스트 초기화
folders_with_few_files = []

# '101'부터 '110'까지의 폴더를 탐색
for folder_num in range(101, 106):
    target_folder = os.path.join(root_dir, str(folder_num))
    if os.path.exists(target_folder):
        # 하위 폴더 탐색
        for food_folder in os.listdir(target_folder):
            food_folder_path = os.path.join(target_folder, food_folder)
            if os.path.isdir(food_folder_path):
                # 해당 폴더 내의 파일 개수 계산
                file_count = glob.glob(os.path.join(food_folder_path, '*'))
                file_count_per_folder[food_folder] = len(file_count)
                # 파일 개수가 1000개 이하인 폴더를 리스트에 추가
                if len(file_count) < 900:
                    folders_with_few_files.append((food_folder, len(file_count)))

    print(len(os.listdir(target_folder)))

# 파일 개수 정보 출력
print("파일 개수 정보:")
for folder, count in file_count_per_folder.items():
    print(f"{folder}: {count}개 파일")

# 1000개 이하의 파일을 가진 폴더 정보 출력
print("\n1000개 이하의 파일을 가진 폴더:")
for folder, count in folders_with_few_files:
    print(f"{folder}: {count}개 파일")

# 결과 리스트 확인
print("\n1000개 이하의 파일을 가진 폴더 리스트:")
print(folders_with_few_files)

In [ ]:
import os
import csv

# 주어진 루트 디렉토리 경로
root_dir = '/content/drive/MyDrive/food_data/Crop'

# CSV 파일 경로
csv_file_path = '/content/drive/MyDrive/food_data/food_structure.csv'

# 파일 경로 정보를 저장할 리스트 초기화
file_structure = []

# 디렉토리를 재귀적으로 탐색하는 함수
def explore_directory(current_dir):
    for root, dirs, files in os.walk(current_dir):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            # 하위 디렉토리 탐색
            explore_directory(dir_path)
        for file_name in files:
            file_path = os.path.join(root, file_name)
            # 폴더 이름과 파일 경로를 리스트에 추가
            food_folder = os.path.basename(root)
            file_structure.append([food_folder, file_path])

# '101'부터 '110'까지의 폴더를 탐색
for folder_num in range(101, 111):
    target_folder = os.path.join(root_dir, str(folder_num))
    if os.path.exists(target_folder):
        explore_directory(target_folder)

# CSV 파일로 저장
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    # 헤더 작성
    writer.writerow(['Food Folder', 'File Path'])
    # 파일 구조 작성
    writer.writerows(file_structure)

print(f"CSV 파일이 '{csv_file_path}'에 저장되었습니다.")

### 사용할 101-110 파일 리스트 저장

In [ ]:
import os
import csv

# 주어진 루트 디렉토리 경로
root_dir = '/content/drive/MyDrive/food_data/Crop'

# CSV 파일 경로
csv_file_path = '/content/drive/MyDrive/food_data/food_structure.csv'

# 파일 경로 정보를 저장할 리스트 초기화
file_structure = []

# 디렉토리를 재귀적으로 탐색하는 함수
def explore_directory(current_dir):
    for root, dirs, files in os.walk(current_dir):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            # 하위 디렉토리 탐색
            explore_directory(dir_path)
        for file_name in files:
            file_path = os.path.join(root, file_name)
            # 폴더 이름과 파일 경로를 리스트에 추가
            food_folder = os.path.basename(root)
            file_structure.append([food_folder, file_path])

# '101'부터 '110'까지의 폴더를 탐색
for folder_num in range(101, 111):
    target_folder = os.path.join(root_dir, str(folder_num))
    if os.path.exists(target_folder):
        explore_directory(target_folder)

# CSV 파일로 저장
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    # 헤더 작성
    writer.writerow(['Food Folder', 'File Path'])
    # 파일 구조 작성
    writer.writerows(file_structure)

print(f"CSV 파일이 '{csv_file_path}'에 저장되었습니다.")